In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2021-07-19 00:50:49,42.6001,-73.9662,68.61,95,100,4.90
1,1,Dubbo,AU,2021-07-19 00:52:24,-32.2500,148.6167,51.93,62,75,9.22
2,2,Busselton,AU,2021-07-19 00:51:38,-33.6500,115.3333,49.05,78,100,8.93
3,3,Karagay,RU,2021-07-19 00:52:25,58.2667,54.9368,58.46,90,0,3.71
4,4,Barcelona,ES,2021-07-19 00:48:08,41.3888,2.1590,79.61,67,20,4.00


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))

In [7]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 87
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
79,79,Fortuna,US,2021-07-19 00:52:44,40.5982,-124.1573,88.20,74,1,3.00
156,156,Todos Santos,MX,2021-07-19 00:53:05,23.4500,-110.2167,88.30,89,73,2.28
271,271,Baghdad,IQ,2021-07-19 00:52:39,33.3406,44.4009,87.71,25,0,6.91
285,285,Umi,JP,2021-07-19 00:53:39,33.5667,130.5000,87.66,40,75,3.44
296,296,Minab,IR,2021-07-19 00:53:42,27.1467,57.0801,89.10,72,50,1.59
340,340,Dalnegorsk,RU,2021-07-19 00:53:55,44.5604,135.5701,89.67,51,0,3.09
354,354,Fountain,US,2021-07-19 00:53:59,38.6822,-104.7008,89.06,39,1,4.61
428,428,Portland,US,2021-07-19 00:53:20,45.5234,-122.6762,89.56,36,0,4.00
437,437,Sioux Lookout,CA,2021-07-19 00:54:22,50.1001,-91.9170,87.87,33,40,8.05
451,451,Hasaki,JP,2021-07-19 00:54:26,35.7333,140.8333,88.29,80,8,11.32


In [14]:
preferred_cities_df.count()

City_ID       14
City          14
Country       14
Date          14
Lat           14
Lng           14
Max Temp      14
Humidity      14
Cloudiness    14
Wind Speed    14
dtype: int64

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
79,Fortuna,US,88.20,40.5982,-124.1573,
156,Todos Santos,MX,88.30,23.4500,-110.2167,
271,Baghdad,IQ,87.71,33.3406,44.4009,
285,Umi,JP,87.66,33.5667,130.5000,
296,Minab,IR,89.10,27.1467,57.0801,
340,Dalnegorsk,RU,89.67,44.5604,135.5701,
354,Fountain,US,89.06,38.6822,-104.7008,
428,Portland,US,89.56,45.5234,-122.6762,
437,Sioux Lookout,CA,87.87,50.1001,-91.9170,
451,Hasaki,JP,88.29,35.7333,140.8333,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
79,Fortuna,US,88.20,40.5982,-124.1573,Super 8 by Wyndham Fortuna
156,Todos Santos,MX,88.30,23.4500,-110.2167,Hotel California Todos Santos
271,Baghdad,IQ,87.71,33.3406,44.4009,Baghdad Hotel
285,Umi,JP,87.66,33.5667,130.5000,Hotel AZ Fukuoka Kanenokuma
296,Minab,IR,89.10,27.1467,57.0801,Bam campground
340,Dalnegorsk,RU,89.67,44.5604,135.5701,Dalnegorsk
354,Fountain,US,89.06,38.6822,-104.7008,Colorado Springs KOA Holiday
428,Portland,US,89.56,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
437,Sioux Lookout,CA,87.87,50.1001,-91.9170,Knobby's Fly-In Camps
451,Hasaki,JP,88.29,35.7333,140.8333,Hotel Sunrise Choshi


In [27]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [31]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [32]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))